Onymos Hiring

Stock Trading Engine

A simple real-time stock trading engine that matches buy and sell orders based on price and quantity.

Features

	•	Supports up to 1,024 stock tickers.  
	•	Handles real-time buy and sell orders.
	•	Matches orders if buy price ≥ lowest sell price.
	•	Ensures thread safety using locks for concurrency.
	•	Uses O(n) time complexity for efficient matching.
	•	Does not use dictionaries, maps, or other high-level data structures.

⸻

How It Works

1.	Placing Orders
	•	A buy order is added if a trader wants to purchase a stock at a specific price.
	•	A sell order is added if a trader wants to sell a stock at a specific price.
	•	Orders are stored in sorted lists for quick matching.



2.	Matching Logic
	•	Orders are sorted so that highest buy prices and lowest sell prices are processed first.
	•	A match occurs when:
  Buy Price >= Lowest Sell Price•	If a match is found, a transaction occurs for the minimum quantity available.
	•	Orders are updated dynamically to reflect the matched quantity.



3.	Handling Concurrency
	•	Uses threading locks to prevent race conditions when multiple traders place orders simultaneously.
	•	Ensures safe order processing in a multi-threaded environment.

In [ ]:
import threading
import random

class TradeOrder:
    def __init__(self, order_kind, stock_symbol, amount, rate):
        self.order_kind = order_kind  # "Buy" or "Sell"
        self.stock_symbol = stock_symbol
        self.amount = amount
        self.rate = rate

class TradingSystem:
    def __init__(self):
        self.buy_list = []  # List to store buy orders
        self.sell_list = []  # List to store sell orders
        self.lock = threading.Lock()  # Lock to prevent race conditions

    def placeOrder(self, order_kind, stock_symbol, amount, rate):
        order = TradeOrder(order_kind, stock_symbol, amount, rate)
        print(f"New Order: {order_kind} {amount} shares of {stock_symbol} at ${rate}")  # Debug Output

        self.lock.acquire()
        try:
            if order_kind == "Buy":
                self.buy_list.append(order)
            else:
                self.sell_list.append(order)
        finally:
            self.lock.release()

        self.processOrders()

    def processOrders(self):
        # Sorting: Highest buy price first, lowest sell price first
        self.buy_list.sort(key=lambda x: (-x.rate, x.stock_symbol))
        self.sell_list.sort(key=lambda x: (x.rate, x.stock_symbol))

        b, s = 0, 0  # Pointers for buy and sell lists

        while b < len(self.buy_list) and s < len(self.sell_list):
            buy_order = self.buy_list[b]
            sell_order = self.sell_list[s]

            if buy_order.stock_symbol != sell_order.stock_symbol or buy_order.rate < sell_order.rate:
                s += 1
                continue

            # Execute transaction
            executed_amount = min(buy_order.amount, sell_order.amount)
            buy_order.amount -= executed_amount
            sell_order.amount -= executed_amount

            print(f"Matched {executed_amount} shares of {buy_order.stock_symbol} at ${sell_order.rate}")

            # Remove completed orders
            if buy_order.amount == 0:
                b += 1
            if sell_order.amount == 0:
                s += 1

        # Retaining only pending orders
        self.lock.acquire()
        try:
            self.buy_list = [o for o in self.buy_list if o.amount > 0]
            self.sell_list = [o for o in self.sell_list if o.amount > 0]
        finally:
            self.lock.release()

def runSimulation(market):
    symbols = ["AAPL", "GOOG", "TSLA", "MSFT"]  # Limited stocks for better matching
    types = ["Buy", "Sell"]

    # Adding forced matching orders
    market.placeOrder("Buy", "AAPL", 50, 150)
    market.placeOrder("Sell", "AAPL", 50, 150)

    market.placeOrder("Buy", "GOOG", 30, 2800)
    market.placeOrder("Sell", "GOOG", 30, 2799)  # Should not match, different price

    # Generating random transactions
    for _ in range(10):  # Reduce to 10 for readability
        order_kind = random.choice(types)
        stock = random.choice(symbols)
        amount = random.randint(1, 50)
        rate = round(random.uniform(100, 3000), 2)  # Keep price range wider
        market.placeOrder(order_kind, stock, amount, rate)

    print("\nUnmatched Orders:")
    print(f"Remaining Buy Orders: {[f'{o.stock_symbol} {o.amount}@${o.rate}' for o in market.buy_list]}")
    print(f"Remaining Sell Orders: {[f'{o.stock_symbol} {o.amount}@${o.rate}' for o in market.sell_list]}")

if __name__ == "__main__":
    market = TradingSystem()
    runSimulation(market)

New Order: Buy 50 shares of AAPL at $150
New Order: Sell 50 shares of AAPL at $150
Matched 50 shares of AAPL at $150
New Order: Buy 30 shares of GOOG at $2800
New Order: Sell 30 shares of GOOG at $2799
Matched 30 shares of GOOG at $2799
New Order: Sell 23 shares of AAPL at $2339.32
New Order: Sell 10 shares of MSFT at $2651.06
New Order: Buy 31 shares of MSFT at $2106.87
New Order: Buy 6 shares of MSFT at $987.91
New Order: Sell 46 shares of GOOG at $947.47
New Order: Sell 20 shares of MSFT at $1131.84
Matched 20 shares of MSFT at $1131.84
New Order: Sell 37 shares of GOOG at $854.81
New Order: Sell 13 shares of TSLA at $2503.57
New Order: Buy 48 shares of TSLA at $2021.57
New Order: Sell 25 shares of GOOG at $1542.71

Unmatched Orders:
Remaining Buy Orders: ['MSFT 11@$2106.87', 'TSLA 48@$2021.57', 'MSFT 6@$987.91']
Remaining Sell Orders: ['GOOG 37@$854.81', 'GOOG 46@$947.47', 'GOOG 25@$1542.71', 'AAPL 23@$2339.32', 'TSLA 13@$2503.57', 'MSFT 10@$2651.06']
